## 使用dataclass

1. 当我们不使用dataclass时，我们需要定义一个类，然后定义__init__方法，然后定义__repr__方法，这样很麻烦,尤其是这个数据需要增加数据时，你需要重写以下所有的魔法方法

In [5]:
class ManualComment:
    def __init__(self, name: str, email: str, comment: str):
        self.name = name
        self.email = email
        self.comment = comment

    def __repr__(self):
        return f'{self.__class__.__name__}(name={self.name!r}, email={self.email!r}, comment={self.comment!r})'

    def __eq__(self, other):
        if not isinstance(other, ManualComment):
            return NotImplemented
        return self.name == other.name and self.email == other.email and self.comment == other.comment

    def __ne__(self, other):
        result = self.__eq__(other)
        if result is NotImplemented:
            return NotImplemented
        return not result

comment = ManualComment('name', 'email', 'comment')
print(comment)

ManualComment(name='name', email='email', comment='comment')


2. 使用dataclass，我们只需要定义一个类，然后使用dataclass装饰器，然后定义字段，这样就可以自动生成__init__方法和__repr__方法
3. [dataclass官方文档](https://docs.python.org/zh-cn/3.10/library/dataclasses.html?highlight=dataclass#module-dataclasses)

In [18]:
from dataclasses import dataclass
import dataclasses
from pprint import pprint
import inspect

@dataclass(frozen=True, order=True) # frozen的含义是冻结该类，不可变，order的含义是可以排序
class Comment:
    name: str
    email: str
    comment: str

comment1 = Comment('Bob', '1234@gmail.com', 'ABCD')
comment2 = Comment('Alice', '45678@gmail.com', 'EFGH')
comment3 = Comment('Bob', '1234@gmail.com', 'ABCD')
print(comment1 == comment2)
print(comment1 == comment3)
pprint(sorted([comment1, comment2, comment3]))
pprint(inspect.getmembers(comment1, inspect.ismethod)) # 打印所有方法
# 当使用frozen=True时，会自动生成__hash__方法，我们可以通过dataclass.replace方法来修改字段的值，但是不会修改字段的值
print(dataclasses.replace(comment1, name='Tom'))
print(comment1)

False
True
[Comment(name='Alice', email='45678@gmail.com', comment='EFGH'),
 Comment(name='Bob', email='1234@gmail.com', comment='ABCD'),
 Comment(name='Bob', email='1234@gmail.com', comment='ABCD')]
[('__delattr__',
  <bound method Comment.__delattr__ of Comment(name='Bob', email='1234@gmail.com', comment='ABCD')>),
 ('__eq__',
  <bound method Comment.__eq__ of Comment(name='Bob', email='1234@gmail.com', comment='ABCD')>),
 ('__ge__',
  <bound method Comment.__ge__ of Comment(name='Bob', email='1234@gmail.com', comment='ABCD')>),
 ('__gt__',
  <bound method Comment.__gt__ of Comment(name='Bob', email='1234@gmail.com', comment='ABCD')>),
 ('__hash__',
  <bound method Comment.__hash__ of Comment(name='Bob', email='1234@gmail.com', comment='ABCD')>),
 ('__init__',
  <bound method Comment.__init__ of Comment(name='Bob', email='1234@gmail.com', comment='ABCD')>),
 ('__le__',
  <bound method Comment.__le__ of Comment(name='Bob', email='1234@gmail.com', comment='ABCD')>),
 ('__lt__',
  <boun

### field方法的使用
大多数时候，对于简单常见的用途，前述的功能已经足够了。而有些功能需要字段提供额外的信息来启用。为了满足这种对附加信息的需求，你可以通过调用提供的 field() 函数来替换字段默认值。例如：

In [19]:
from dataclasses import dataclass, field
@dataclass
class C:
    name: str = field(
        default='Tom',
        metadata={'help': 'The name of the person'}
    )
    email: str = field(default="")
    comment: str = field(default='Hello')
    replies: list[str] = field(
        default_factory=list,
        metadata={'help': 'The replies for the comment'},
        compare=False
    )



In [25]:
# 如果不使用field方法，默认值会出现一些问题，例如下面的例子，replies的默认值是一个空列表，但是当我们修改一个实例的replies时，其他实例的replies也会被修改
# 这是因为默认值是一个可变对象，当我们修改一个实例的replies时，实际上是修改了这个可变对象，所以其他实例的replies也会被修改
# 因此要么默认值设置为None，要么使用field方法
@dataclass
class C:
    name: str = 'Tom'
    email: str = ""
    comment: str = 'Hello'
    replies = []
c1 = C()
c2 = C()
c1.replies.append('reply1') # 修改c1的replies
print(c1.replies)
print(c2.replies) # c2的replies也被修改了
print(id(c1.replies))
print(id(c2.replies))

['reply1']
['reply1']
2165412545280
2165412545280


### 题外话：NotImplemented
NotImplemented是 Python 内置命名空间中的六个常量之一。其他还有 False, True, None, Ellipsis 和 \_\_debug\_\_,NotImplemented是 python 特殊二元方法（例如__eq__(), __lt__(), __add__(), __rsub__()）返回的特殊值，表示该操作没有针对其他类型实现。而且，它转换成 bool 类型表示 true